In [1]:
from keras.models import Sequential
from keras.layers import Conv2D,ZeroPadding2D,Activation,Input,concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D,AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda,Flatten,Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

np.set_printoptions(threshold=np.nan)


Using TensorFlow backend.
C:\Users\CHENwenyu\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\CHENwenyu\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\CHENwenyu\anaconda3\envs\tensorflow\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\CHENwenyu\a

ValueError: threshold must be numeric and non-NAN, try sys.maxsize for untruncated representation

In [5]:
FRmodel=faceRecoModel(input_shape=(3,96,96))


In [ ]:
print('total params:',FRmodel.count_params())

total params: 3743280


In [2]:
def triplet_loss(y_true,y_pred,alpha=0.2):
    anchor,positive,negative=y_pred[0],y_pred[1],y_pred[2]

    pos_dist=tf.reduce_sum(tf.square(tf.subtract(anchor,positive)))

    neg_dist=tf.reduce_sum(tf.square(tf.subtract(anchor,negative)))

    basic_loss=tf.add(tf.subtract(pos_dist,neg_dist),alpha)

    loss=tf.reduce_sum(tf.maximum(basic_loss,0.0))

    return loss

In [3]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true=(None,None,None)
    y_pred=(tf.random_normal([3,128],mean=6,stddev=0.1,seed=1),
            tf.random_normal([3,128],mean=1,stddev=1,seed=1),
            tf.random_normal([3,128],mean=3,stddev=4,seed=1))

    loss=triplet_loss(y_true,y_pred)

    print('loss='+str(loss.eval()))

loss=350.02716


In [7]:
FRmodel.compile(optimizer='adam',loss=triplet_loss,metrics=['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [8]:
database = {}
database["danielle"] = img_to_encoding("images/danielle.png", FRmodel)
database["younes"] = img_to_encoding("images/younes.jpg", FRmodel)
database["tian"] = img_to_encoding("images/tian.jpg", FRmodel)
database["andrew"] = img_to_encoding("images/andrew.jpg", FRmodel)
database["kian"] = img_to_encoding("images/kian.jpg", FRmodel)
database["dan"] = img_to_encoding("images/dan.jpg", FRmodel)
database["sebastiano"] = img_to_encoding("images/sebastiano.jpg", FRmodel)
database["bertrand"] = img_to_encoding("images/bertrand.jpg", FRmodel)
database["kevin"] = img_to_encoding("images/kevin.jpg", FRmodel)
database["felix"] = img_to_encoding("images/felix.jpg", FRmodel)
database["benoit"] = img_to_encoding("images/benoit.jpg", FRmodel)
database["arnaud"] = img_to_encoding("images/arnaud.jpg", FRmodel)

In [10]:
def verify(img_path,identity,database,model):
    encoding=img_to_encoding(img_path,model)

    dist=np.linalg.norm(encoding-database[identity])

    if dist<0.7:
        print("It's " + str(identity) + ", welcome home!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
    
    return dist,door_open

    

In [11]:
verify('images/camera_0.jpg','younes',database,FRmodel)


It's younes, welcome home!


(0.66714054, True)

In [12]:
verify("images/camera_2.jpg", "kian", database, FRmodel)

It's not kian, please go away


(0.8586888, False)

In [13]:
def who_is_it(image_path,database,model):
    encoding=img_to_encoding(image_path,model)

    min_dist=100

    for (name,db_enc) in database.items():

        dist=np.linalg.norm(encoding-db_enc)

        if dist<min_dist:

            min_dist=dist
            identity=name
    
    if min_dist>0.7:
        print('Not in the database.')

    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist,identity
    

In [14]:
who_is_it('images/camera_0.jpg',database,FRmodel)

it's younes, the distance is 0.66714054


(0.66714054, 'younes')

In [15]:
who_is_it('images/camera_1.jpg',database,FRmodel)

it's bertrand, the distance is 0.46807337


(0.46807337, 'bertrand')

In [16]:
who_is_it('images/camera_2.jpg',database,FRmodel)

it's benoit, the distance is 0.2283746


(0.2283746, 'benoit')